## SEARCH ENGINE WITH TOOLS AND AGENTS

In [1]:
## ARXIV -- RESEARCH PAPER
## TOOLS CREATION
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [2]:
## USED INBUILT TOOLS OF WIKIPEDIA
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)   ## THIS HELPS TO SEARCH AND FETECH DATA FROM WIKIPEDIA
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [3]:
## USED INBUILT TOOLS OF ARXIV
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_classic.chains  import create_retrieval_chain
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_community.vectorstores import FAISS

c:\Users\hellh\Machine Learning\LangChain\venc\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
groq_api_key = os.getenv('GROQ_API_KEY')
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
os.environ["LANGCHAIN_TRACING_V2"] = "true"  
os.environ["LANGCHAIN_PROJECT"] = "Search Engine With Tools And Agents"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

In [8]:
llm = ChatGroq(model="llama-3.1-8b-instant",groq_api_key=groq_api_key)
embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")

In [9]:
loader = WebBaseLoader("https://docs.langchain.com/langsmith/home")
docs = loader.load()
dcouments = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents=dcouments,embedding=embeddings)
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002A4B0530490>, search_kwargs={})

In [10]:
## CUSTOM TOOLS [RAG TOOLS] -- [langsmith_search]
from langchain_core.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith")
retriever_tool.name

'langsmith-search'

In [11]:
## ADD CUSTOM TOOLS IN THE LIST
tools = [wiki,arxiv,retriever_tool]

In [83]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\hellh\\Machine Learning\\LangChain\\venc\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000025CD297E9E0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x

In [84]:
## RUN THESE TOOLS WITH AGENT AND LLM MODELS
## COMBINE THESE TOOLS WITH LLM MODELS ALONG WITH AGENT EXECUTOR

In [59]:
## PROMPT TEMPLATE
from langchain_classic import hub                                   ## HUB IS TO USED TO EXTRACT PRE WRITTEN PROMPT INSTEAD OF WRITTING IT MANUALLY
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [ ]:
## CREATE AGENTS
from langchain_classic.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [61]:
## EXECUTE AGENT
from langchain_classic.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [62]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'about Langsmith'}`


It helps you trace requests, evaluate outputs, test prompts, and manage deployments in one place.
LangSmith is framework agnostic, so you can use it with or without LangChain’s open-source libraries
langchain and langgraph.
Prototype locally, then move to production with integrated monitoring and evaluation to build more reliable AI systems.
LangGraph Platform is now LangSmith Deployment. For more information, check out the Changelog.
​Get started
Create an accountSign up at smith.langchain.com (no credit card required).
You can log in with Google, GitHub, or email.Create an API keyGo to your Settings page → API Keys → Create API Key.
Copy the key and save it securely.
Once your account and API key are ready, choose a quickstart to begin building with LangSmith:

LangSmith meets the highest standards of data security and privacy with HIPAA, SOC 2 Type 2, and GDPR compliance. For 

{'input': 'Tell me about Langsmith',
 'output': 'It seems like I provided a similar response to your previous question. I will provide a more detailed answer.\n\nTo use LangSmith, you need to follow these steps:\n\n1. Create an account: Go to smith.langchain.com and sign up with your Google, GitHub, or email account.\n2. Create an API key: Go to your Settings page, click on API Keys, and create a new API key. Copy the key and save it securely.\n3. Choose a quickstart: Once your account and API key are ready, choose a quickstart to begin building with LangSmith.\n\nLangSmith provides a workflow that combines observability, evaluation, deployment, and platform setup in one integrated workflow. This allows you to prototype locally and then move to production with integrated monitoring and evaluation to build more reliable AI systems.\n\nSome of the key features of LangSmith include:\n\n* Observability: Gain visibility into every step your application takes to debug faster and improve reli

In [64]:
agent_executor.invoke({"input":"What is machine learning"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'machine learning'}`


Published: 2023-06-08
Title: Changing Data Sources in the Age of Machine Learning for Official Statistics
Authors: Cedric De Boom, Michael Reusens
Summary: Data science has become increasingly essential for the production of official statistics, as i
Invoking: `wikipedia` with `{'query': 'machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks witIt appears that the information provided by the functions does not fully answer the question. However, based on the results of the functions, here is a revised answer:

Machine learning is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise 

{'input': 'What is machine learning',
 'output': 'It appears that the information provided by the functions does not fully answer the question. However, based on the results of the functions, here is a revised answer:\n\nMachine learning is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks with some degree of autonomy.'}